<a href="https://colab.research.google.com/github/brandonliu12/nfl-qb-prediction/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries, read files, and clean it further

In [174]:
import pandas as pd
path = 'NFL Play by Play 2009-2018 (v5).csv'
df = pd.read_csv(path)

df['game_date'] = pd.to_datetime(df['game_date'])
df['year'] = df['game_date'].dt.year
df_2018 = df[df['game_date'].dt.year == 2018]
df = df[df['year'] != 2018]



<ipython-input-174-31ba2dc60365>:3: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [175]:
df_2018.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,year
407335,37,2018090600,PHI,ATL,ATL,away,PHI,PHI,35.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018
407336,52,2018090600,PHI,ATL,ATL,away,PHI,ATL,75.0,2018-09-06,...,L.Paulsen,5.0,0,NaN,False Start,0.0,0.0,0.0,0.0,2018
407337,75,2018090600,PHI,ATL,ATL,away,PHI,ATL,80.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018
407338,104,2018090600,PHI,ATL,ATL,away,PHI,ATL,70.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018
407339,125,2018090600,PHI,ATL,ATL,away,PHI,ATL,59.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018


In [176]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['passer_player_name_encoded'] = encoder.fit_transform(df['passer_player_name'])

In [177]:
import numpy as np
relevant_columns = ['yardline_100','game_seconds_remaining','qtr', 'down', 'air_yards', 'pass_attempt', 'posteam_timeouts_remaining',
                     'complete_pass', 'passer_player_name_encoded', 'pass_location', 'score_differential_post', 'posteam_type', 'half_seconds_remaining']

# Select the data using the relevant columns
df_selected = df[relevant_columns]

# Get list of categorical columns
categorical_cols = df_selected.select_dtypes(include=['object']).columns

# Create dummy variables for all categorical columns
df_dummies = pd.get_dummies(df_selected[categorical_cols])

# Combine original numerical data with dummy variables
df_combined = pd.concat([df_selected.select_dtypes(include=[np.number]), df_dummies], axis=1)

df_combined = df_combined.fillna(0)

df_combined.head()

,yardline_100,game_seconds_remaining,qtr,down,air_yards,pass_attempt,posteam_timeouts_remaining,complete_pass,passer_player_name_encoded,score_differential_post,half_seconds_remaining,pass_location_left,pass_location_middle,pass_location_right,posteam_type_away,posteam_type_home
0,30.0,3600.0,1,0.0,0.0,0.0,3.0,0.0,346,0.0,1800.0,0,0,0,0,1
1,58.0,3593.0,1,1.0,-3.0,1.0,3.0,1.0,47,0.0,1793.0,1,0,0,0,1
2,53.0,3556.0,1,2.0,0.0,0.0,3.0,0.0,346,0.0,1756.0,0,0,0,0,1
3,56.0,3515.0,1,3.0,34.0,1.0,3.0,0.0,47,0.0,1715.0,0,0,1,0,1
4,56.0,3507.0,1,4.0,0.0,0.0,3.0,0.0,346,0.0,1707.0,0,0,0,0,1


In [178]:
df_2018['passer_player_name_encoded'] = encoder.fit_transform(df_2018['passer_player_name'])

df_selected_2018 = df_2018[relevant_columns + ['play_id']]

# Get list of categorical columns
categorical_cols = df_selected_2018.select_dtypes(include=['object']).columns

# Create dummy variables for all categorical columns in 2018 dataframe
df_dummies_2018 = pd.get_dummies(df_selected_2018[categorical_cols])

# Combine original numerical data with dummy variables for 2018 dataframe
df_combined_2018 = pd.concat([df_selected_2018.select_dtypes(include=[np.number]), df_dummies_2018, df_selected_2018['play_id']], axis=1)

# Fill NaN values with 0 for 2018 dataframe
df_combined_2018 = df_combined_2018.fillna(0)
df_2018 = df_2018.fillna(0)


df_combined_2018.head()

,yardline_100,game_seconds_remaining,qtr,down,air_yards,pass_attempt,posteam_timeouts_remaining,complete_pass,passer_player_name_encoded,score_differential_post,half_seconds_remaining,play_id,pass_location_left,pass_location_middle,pass_location_right,posteam_type_away,posteam_type_home,play_id
407335,35.0,3600.0,1,0.0,0.0,0.0,3.0,0.0,98,0.0,1800.0,37,0,0,0,1,0,37
407336,75.0,3600.0,1,1.0,0.0,0.0,3.0,0.0,98,0.0,1800.0,52,0,0,0,1,0,52
407337,80.0,3600.0,1,1.0,8.0,1.0,3.0,1.0,69,0.0,1800.0,75,0,0,1,1,0,75
407338,70.0,3562.0,1,2.0,0.0,0.0,3.0,0.0,98,0.0,1762.0,104,0,0,0,1,0,104
407339,59.0,3526.0,1,1.0,0.0,0.0,3.0,0.0,98,0.0,1726.0,125,0,0,0,1,0,125


Machine Learning Model

In [179]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Define target variable
target = df_combined['air_yards']

# Define features
features = df_combined.drop('air_yards', axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [180]:
from sklearn.ensemble import RandomForestRegressor

# Create a random forest regressor object
random_forest = RandomForestRegressor(random_state=42, n_estimators=100)

# Train the model using the training sets
random_forest.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = random_forest.predict(X_test)


In [181]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f"Model RMSE: {rmse}")


Model RMSE: 6.09647459012335


In [182]:
mean_air_yards = df['air_yards'].mean()
mean_air_yards

8.345890389457095

Predict 2018 performance

In [183]:
# Separate the features from the target variable in df_2018
features_2018 = df_combined_2018.drop(['air_yards', 'play_id'], axis=1)

# Use the trained model to predict 'air_yards' for df_2018
predictions_2018 = random_forest.predict(features_2018)

# Add the predictions as a new column in df_2018
df_2018['predicted_air_yards'] = predictions_2018

<ipython-input-183-65e47ed5e1e5>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2018['predicted_air_yards'] = predictions_2018


In [184]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separate the actual 'air_yards' values
actuals_2018 = df_2018['air_yards']

# Calculate MAE, MSE, RMSE, and R-squared
mae = mean_absolute_error(actuals_2018, predictions_2018)
mse = mean_squared_error(actuals_2018, predictions_2018)
rmse = np.sqrt(mse)  # or directly use mean_squared_error(actuals_2018, predictions_2018, squared=False)
r2 = r2_score(actuals_2018, predictions_2018)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")


MAE: 2.9705644354159024
MSE: 37.32423559847764
RMSE: 6.109356398056807
R-squared: 0.3247805683566587


In [185]:
predicted_df = df_2018[['air_yards', 'predicted_air_yards']]

predicted_df['pass_distance_type'] = 'short'  # default value
predicted_df.loc[predicted_df['air_yards'] > 20, 'pass_distance_type'] = 'long'
predicted_df['predicted_pass_distance_type'] = 'short'  # default value
predicted_df.loc[predicted_df['predicted_air_yards'] > 20, 'pass_distance_type'] = 'long'
predicted_df.head(50)


<ipython-input-185-00f0d93ca01e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df['pass_distance_type'] = 'short'  # default value
<ipython-input-185-00f0d93ca01e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df['predicted_pass_distance_type'] = 'short'  # default value


,air_yards,predicted_air_yards,pass_distance_type,predicted_pass_distance_type
407335,0.0,0.000,short,short
407336,0.0,0.000,short,short
407337,8.0,4.158,short,short
407338,0.0,0.000,short,short
407339,0.0,0.000,short,short
407340,4.0,10.820,short,short
407341,-3.0,11.270,short,short
407342,24.0,7.250,long,short
407343,0.0,0.000,short,short
407344,0.0,-0.440,short,short


Continue predicting additional statistic such as pass location and distance